# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-a0d97ac515-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Euro explorative search") 

Consider the following exploratory information need:

> investigate the Euro currency, which are the countries that currently use the Euro and which were their currencies before the Euro. Take a look also to other currencies and find which are the most used.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q38`| Italy   | node |





Also consider

```
wd:Q38 ?p ?obj .
```

is the BGP to retrieve all **properties of Italy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for currency

2. Return the countries which currently use Euro

3. Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

4. For each country that now uses Euro return which was the currencies used before

5. Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

6. Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

7. Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

## Task 1
Identify the BGP for currency

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [3]:
# ran DESCRIBE before realizing it should not be used..
queryString = """
DESCRIBE wd:P1198
WHERE {
   wd:Q38 ?p wd:P1198 .
   # get the label

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http://www.wikidata.org/prop/direct/P2302'), ('o', 'http://www.wikidata.org/entity/Q21510860')]
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http://www.wikidata.org/prop/direct/P1896'), ('o', 'http://data.worldbank.org/indicator/SL.UEM.TOTL.ZS')]
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http://wikiba.se/ontology#statementValue'), ('o', 'http://www.wikidata.org/prop/statement/value/P1198')]
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http://wikiba.se/ontology#novalue'), ('o', 'http://www.wikidata.org/prop/novalue/P1198')]
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http://wikiba.se/ontology#directClaimNormalized'), ('o', 'http://www.wikidata.org/prop/direct-normalized/P1198')]
[('s', 'http://www.wikidata.org/entity/P9396'), ('p', 'http://www.wikidata.org/prop/direct/P1659'), ('o', 'http://www.wikidata.org/entity/P1198')]
[('s', 'http://www.wikidata.org/entity/P1198'), ('p', 'http

/home/ubuntu/miniconda3/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:979: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn("Sending Accept header '*/*' because unexpected returned format '%s' in a '%s' SPARQL query form" % (self.returnFormat, self.queryType), RuntimeWarning)


In [4]:
# I want to check if unemployment rate has some connections to money
# Actually unemployment rate is a predicate, so wd:P1198 is something else. -> Didn't realize it in the beginning

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:P1198 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q24710760'), ('name', 'Wikidata property of quantity type indicating a percentage')]
[('obj', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('obj', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('obj', 'http://www.wikidata.org/entity/Q19474404'), ('name', 'single-value constraint')]
[('obj', 'http://www.wikidata.org/entity/Q21503250'), ('name', 'type constraint')]
[('obj', 'http://www.wikidata.org/entity/Q52004125'), ('name', 'allowed entity types constraint')]
[('obj', 'http://www.wikidata.org/entity/Q53869507'), ('name', 'property scope constraint')]
[('obj', 'http://www.wikidata.org/entity/Q54554025'), ('name', 'citation needed constraint')]
[('obj', 'http://www.wikidata.org/entity/Q21451178'), ('name', 'Wikidata property related to economics')]
[('obj', 'http://www.wikidata.org/entity/Q21510860'), ('nam

In [5]:
# see what are the outgoing edges from Italy. same as initial query, increasing limit
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [6]:
# FINAL QUERY
# every country has a currency, so Italy should have one.
# This is the predicate "currency", used thoughout the rest of the notebook.

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name, "(i?)currency"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
1


>wdt:P38 is the property with label "currency"

In [7]:
# every country has a currency, so Italy should have one. 
# Now I am looking for node "currency",for completeness.

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q38 ?p ?obj .
   # get the label
   #?p sc:name ?name.
   ?obj sc:name ?objName.
   FILTER(REGEX(?objName, "(i?)currency"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
# Italy and Currency should be connected. Every country has a currency.
# Here I am looking for path of length up to 3 from node Italy to label "currency"
# Iteratively decreased path length and LIMIT to 2.
# wd:Q8142 is the node with label "currency"


# !!!!
# Since I am asking for any path, it can be inefficient and take long to compute. I am doing it and not looking for the exact path
# only because I found the node currency in later tasks. 
# !!!!

queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {{
   # bind something
   wd:Q38 (<>|!<>){,2} ?iri.
   # get the label
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)currency"))
   FILTER (!isLiteral(?iri))
   }UNION{
   ?iri (<>|!<>){,2} wd:Q38.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)currency"))
   }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)


Results
[('iri', 'http://www.wikidata.org/entity/Q28795800'), ('iriName', 'decimal currency')]
[('iri', 'http://www.wikidata.org/entity/Q8142'), ('iriName', 'currency')]
2


## Task 2
Return the countries which currently use Euro

In [9]:
# check all subjects that are instances of currency, but empty set. Forgot currency (wdt:P38) is a predicate and not a node...

queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
    ?s wdt:P31 wdt:P38.
   # get the label
   ?s sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
# all incoming edges to currency. Empty because currency (wdt:P38) is a predicate and not a node.

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?s ?p wdt:P38.
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [11]:
# seems I am using the wrong iri wdt:P38 instead of wd:P38?. Retrieving predicates which have 
# currency as subject. Not helpful because wd:P38 is not related with what I am doing, it's not something I found.
queryString = """
SELECT DISTINCT ?p ?pname ?o ?name
WHERE {
   # bind something
     wd:P38 ?p ?o.
     # get the label
     ?o sc:name ?name.
     ?p sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list'), ('o', 'http://www.wikidata.org/entity/Q25022717'), ('name', 'Wikidata:Wikivoyage/Lists/Quickbar')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q21451178'), ('name', 'Wikidata property related to economics')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pname', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/entity/Q747699'), ('name', 'unit of account')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pname', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/entity/Q8142'), ('name', 'currency')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pname', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pname', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q148

In [12]:
# all predicates which have currency as object. Not helpful, because I am using the wrong iri, wd:P38 is not related with what I am doing.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?s ?p wd:P38.
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('name', 'see also')]
3


In [13]:
# try to find objects with subject Italy and predicate currency, but none because using wrong IRI(wd:P38 instead of wdt:P38). 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wd:P38 ?obj .
   # get the label
   ?obj sc:name ?name.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [14]:
# try to find ?obj of italian currency with iri wdt:P38 instead of wd:38.
# I retrieve euro (wd:Q4916)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38 wdt:P38 ?obj .
   # get the label
   ?obj sc:name ?name.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4916'), ('name', 'euro')]
1


In [15]:
# all nodes with subject euro
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q4916 ?p ?obj .
   # get the label
   ?obj sc:name ?name.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16068'), ('name', 'Deutsche Mark')]
[('obj', 'http://www.wikidata.org/entity/Q788472'), ('name', 'Dutch guilder')]
[('obj', 'http://www.wikidata.org/entity/Q203354'), ('name', 'Finnish markka')]
[('obj', 'http://www.wikidata.org/entity/Q270642'), ('name', 'Irish pound')]
[('obj', 'http://www.wikidata.org/entity/Q28795800'), ('name', 'decimal currency')]
[('obj', 'http://www.wikidata.org/entity/Q7778928'), ('name', 'Category:Euro')]
[('obj', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q152337'), ('name', 'Charlemagne Prize')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('obj', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('obj', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('obj', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('obj', 'h

In [16]:
# try to find iri of country by checking all possible predicates-objects with subject Spain to see if country is present
# Too many results
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name
WHERE {
   # bind something
   wd:Q29 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal"), ('obj', 'http://www.wikidata.org/entity/Q3249424'), ('name', 'Portal:Spain')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government'), ('obj', 'http://www.wikidata.org/entity/Q3330103'), ('name', 'parliamentary monarchy')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of government'), ('obj', 'http://www.wikidata.org/entity/Q844587'), ('name', 'Prime Minister of Spain')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q302556'), ('name', 'Catholic Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described

In [17]:
# try to find iri of country by checking all possible predicates with subject Spain to see if country is present
# returning only distinct predicates
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   wd:Q29 ?p ?obj .
   # get the label
   ?p sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('pname', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('pname', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pname', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pname', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pname', 'office held by head of

In [18]:
# check what is Spain "instance of" -> sovereign state but not country
queryString = """
SELECT DISTINCT ?obj ?pname 
WHERE {
   # bind something
   wd:Q29 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('pname', 'sovereign state')]
1


In [19]:
# check what is Italy "instance of".
# Italy is both "instance of" soveregin state and country
queryString = """
SELECT DISTINCT ?obj ?pname 
WHERE {
   # bind something
   wd:Q38 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3624078'), ('pname', 'sovereign state')]
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('pname', 'country')]
2


In [20]:
# take all distinct objects of the predicate "instance of", and subjects being also a country (wd:Q6256).
# Spain is not a country -> to retrieve all actual countries maybe we need to consider more than just wd:Q6256
queryString = """
SELECT DISTINCT ?regions ?name
WHERE {
?s wdt:P31 wd:Q6256;
    wdt:P31 ?regions.
?regions sc:name ?name



}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('regions', 'http://www.wikidata.org/entity/Q1489259'), ('name', 'superpower')]
[('regions', 'http://www.wikidata.org/entity/Q1250464'), ('name', 'realm')]
[('regions', 'http://www.wikidata.org/entity/Q1323642'), ('name', 'transcontinental country')]
[('regions', 'http://www.wikidata.org/entity/Q99541706'), ('name', 'historical unrecognized state')]
[('regions', 'http://www.wikidata.org/entity/Q353344'), ('name', 'countship')]
[('regions', 'http://www.wikidata.org/entity/Q1179466'), ('name', 'de facto regime')]
[('regions', 'http://www.wikidata.org/entity/Q1048835'), ('name', 'political territorial entity')]
[('regions', 'http://www.wikidata.org/entity/Q10742'), ('name', 'autonomous community of Spain')]
[('regions', 'http://www.wikidata.org/entity/Q1088391'), ('name', 'rump state')]
[('regions', 'http://www.wikidata.org/entity/Q112099'), ('name', 'island nation')]
[('regions', 'http://www.wikidata.org/entity/Q118365'), ('name', 'dictatorship of the proletariat')]
[('regions',

In [21]:
# all regions having as currency euro
# not only countries are included
queryString = """
SELECT DISTINCT ?countries ?name
WHERE {
?s wdt:P31 wd:Q6256;
    wdt:P31 ?regions.
?countries wdt:P31 ?regions;
    wdt:P38 wd:Q4916.
        

?countries sc:name ?name


}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('countries', 'http://www.wikidata.org/entity/Q129003'), ('name', 'French Southern and Antarctic Lands')]
[('countries', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('countries', 'http://www.wikidata.org/entity/Q458'), ('name', 'European Union')]
[('countries', 'http://www.wikidata.org/entity/Q5705'), ('name', 'Catalonia')]
[('countries', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('countries', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('countries', 'http://www.wikidata.org/entity/Q11959'), ('name', 'Alicante')]
[('countries', 'http://www.wikidata.org/entity/Q12258'), ('name', 'Sabadell')]
[('countries', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands')]
[('countries', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('countries', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal')]
[('countries', 'http://www.wikidata.org/entity/Q37024'), ('name', 'Serbia and Montenegro')

In [22]:
# Maybe there is a more precise relationship from euro to the countries 
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   wd:Q4916 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   ?p sc:name ?pname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pname', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P489'), ('pname', 'currency symbol description')]
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('pname', 'on focus list of Wikimedia project')]
[('p', 'http://www.wikidata.org/prop/direct/P562'), ('pname', 'central bank/issuer')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's mai

In [23]:
# FINAL QUERY
queryString = """
SELECT DISTINCT ?obj ?name 
WHERE {
   # bind something
   wd:Q4916 wdt:P17 ?obj .
   # get the label
   ?obj sc:name ?name.


}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('obj', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('obj', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('obj', 'http://www.wikidata.org/entity/Q215'), ('name', 'Slovenia')]
[('obj', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('obj', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('obj', 'http://www.wikidata.org/entity/Q233'), ('name', 'Malta')]
[('obj', 'http://www.wikidata.org/entity/Q235'), ('name', 'Monaco')]
[('obj', 'http://www.wikidata.org/entity/Q237'), ('name', 'Vatican City')]
[('obj', 'http://www.wikidata.org/entity/Q238'), ('name', 'San Marino')]
[('obj', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('obj', 'ht

> Results in the format "Country IRI" - "Country Label".


## Task 3
Return the countries belonging to the European Union, with a currency different from the Euro (return also their currency).

In [24]:
# All incoming edges to EU
# wd:Q458 stands for EU, found in a query before with ctrl+f

queryString = """
SELECT DISTINCT ?incoming ?name where {
?s ?incoming wd:Q458.
?incoming sc:name ?name

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('incoming', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('incoming', 'http://www.wikidata.org/prop/direct/P3005'), ('name', 'valid in place')]
[('incoming', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('incoming', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('incoming', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('incoming', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('incoming', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('incoming', 'http://www.wikidata.org/prop/direct/P126'), ('name', 'maintained by')]
[('incoming', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('incoming', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('incoming', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('incoming',

In [25]:
# After reading all 71 results of the previous query I found predicate "member of" P463.
# all members of eu. UK not anymore..
queryString = """
SELECT DISTINCT ?s ?name where {
?s wdt:P463 wd:Q458.
?s sc:name ?name

}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('s', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium')]
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[('s', 'http://www.wikidata.org/entity/Q229'), ('name', 'Cyprus')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark')]
[('s', 'http://www.wikidata.org/entity/Q191'), ('name', 'Estonia')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland')]
[('s', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('s', 'http://www.wikidata.org/entity/Q41'), ('name', 'Greece')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary')]
[('s', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland')]
[('s', 'http://www.wikidata.org/entity/Q38

In [26]:
# FINAL QUERY
# Find all european union members and their currency minus the one that use the euro
queryString = """
SELECT DISTINCT ?s ?name ?currency ?currencyName where {
?s wdt:P463 wd:Q458.
?s wdt:P38 ?currency.
?s sc:name ?name.
?currency sc:name ?currencyName
MINUS{
?s wdt:P38 wd:Q4916.
}
}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('currency', 'http://www.wikidata.org/entity/Q172540'), ('currencyName', 'Bulgarian lev')]
[('s', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia'), ('currency', 'http://www.wikidata.org/entity/Q26360'), ('currencyName', 'Croatian kuna')]
[('s', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic'), ('currency', 'http://www.wikidata.org/entity/Q131016'), ('currencyName', 'Czech koruna')]
[('s', 'http://www.wikidata.org/entity/Q35'), ('name', 'Denmark'), ('currency', 'http://www.wikidata.org/entity/Q25417'), ('currencyName', 'Danish krone')]
[('s', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('currency', 'http://www.wikidata.org/entity/Q47190'), ('currencyName', 'forint')]
[('s', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('currency', 'http://www.wikidata.org/entity/Q123213'), ('currencyName', 'złoty')]
[('s', 'http://www.wikidata.org/entity/Q218'), ('name'

>Results are in the format "Country IRI" - "Country Label" - "Currency" - " Currency Label"

## Task 4
For each country that now uses Euro return which was the currencies used before

In [27]:
# all incoming edges of EU members. Tested with limit 500, not useful

queryString = """
SELECT DISTINCT ?p ?name where {
?member wdt:P463 wd:Q458.
?s ?p ?member.
?p sc:name ?name.
}
LIMIT 1
#LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('name', 'defendant')]
1


In [28]:
# incoming and outgoing edges of Italy. Not useful, tested with limit 50.
queryString = """
SELECT DISTINCT ?incoming ?nameIncoming ?outgoing ?nameOutgoing where {
{wd:Q38 ?outgoing ?object.
?outgoing sc:name ?nameOutgoing.
}
UNION{
?subject ?incoming wd:Q38.
?incoming sc:name ?nameIncoming.
}


}
LIMIT 1
#LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('outgoing', 'http://www.wikidata.org/prop/direct/P1005'), ('nameOutgoing', 'Portuguese National Library ID')]
1


In [29]:
# I searched (CTRL+F) for "lira" in this notebook and found a query before containing italian lira among the results. start from there.
# look for predicate that connects euro with italian lira

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q4916 ?p wd:Q204992.
   # get the label
   ?p sc:name ?name.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
1


In [30]:
# how is Italy connected to it's ancestors (country it replaces)
queryString = """
SELECT DISTINCT ?s ?name ?p ?previousCountryProperty where {

?s wdt:P38 wd:Q204992.
?s ?p wd:Q38.

?s sc:name ?name.
?p sc:name ?previousCountryProperty.

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy'), ('p', 'http://www.wikidata.org/prop/direct/P1366'), ('previousCountryProperty', 'replaced by')]
[('s', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('previousCountryProperty', 'followed by')]
2


In [31]:
# FINAL QUERY
# For each country that now uses euro, find the countries it replaces, returning the old currency

queryString = """
SELECT DISTINCT ?s ?name ?ancestor ?ancestorName ?result ?ancestorCurrency where {
wd:Q4916 wdt:P17 ?s . # "euro"-"country"-"country iri"
?s wdt:P38 ?currency.
?ancestor ?replacedBy ?s.
?ancestor wdt:P38 ?result.

VALUES ?replacedBy {wdt:P1366 wdt:P156}

?s sc:name ?name.
?ancestor sc:name ?ancestorName.
?result sc:name ?ancestorCurrency.


}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra'), ('ancestor', 'http://www.wikidata.org/entity/Q31929'), ('ancestorName', 'Carolingian Empire'), ('result', 'http://www.wikidata.org/entity/Q1620502'), ('ancestorCurrency', 'Catalan peseta')]
[('s', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('ancestor', 'http://www.wikidata.org/entity/Q15864'), ('ancestorName', 'United Kingdom of the Netherlands'), ('result', 'http://www.wikidata.org/entity/Q788472'), ('ancestorCurrency', 'Dutch guilder')]
[('s', 'http://www.wikidata.org/entity/Q33'), ('name', 'Finland'), ('ancestor', 'http://www.wikidata.org/entity/Q1811029'), ('ancestorName', "Finnish Socialist Workers' Republic"), ('result', 'http://www.wikidata.org/entity/Q203354'), ('ancestorCurrency', 'Finnish markka')]
[('s', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('ancestor', 'http://www.wikidata.org/entity/Q16957'), ('ancestorName', 'German Democratic Republic'), ('result', 'http:/

>Results in the format "Country IRI" - "Country Label" - "Ancestor Country IRI" - "Ancestor Country Label" - "Ancestor Currency IRI" - "Ancestor Currency Label"

In [32]:
# same as before, just more readable
queryString = """
SELECT DISTINCT ?name (GROUP_CONCAT(DISTINCT ?ancestorCurrency; separator = ", ") as ?previousCurrencies)  where {

wd:Q4916 wdt:P17 ?s .
?s wdt:P38 ?currency.
?ancestor ?replacedBy ?s.
?ancestor wdt:P38 ?result.

VALUES ?replacedBy {wdt:P1366 wdt:P156}

?s sc:name ?name.
?ancestor sc:name ?ancestorName.
?result sc:name ?ancestorCurrency.


}
ORDER BY ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Andorra'), ('previousCurrencies', 'Catalan peseta')]
[('name', 'Belgium'), ('previousCurrencies', 'Dutch guilder')]
[('name', 'Finland'), ('previousCurrencies', 'Finnish markka')]
[('name', 'Germany'), ('previousCurrencies', 'Deutsche Mark, East German mark')]
[('name', 'Ireland'), ('previousCurrencies', 'Irish pound, pound sterling')]
[('name', 'Italy'), ('previousCurrencies', 'Italian lira')]
[('name', 'Latvia'), ('previousCurrencies', 'Soviet ruble')]
[('name', 'Luxembourg'), ('previousCurrencies', 'Conventionsthaler, Reichsthaler, Vereinsthaler')]
[('name', 'Malta'), ('previousCurrencies', 'Maltese pound, pound sterling')]
[('name', 'Netherlands'), ('previousCurrencies', 'Dutch guilder')]
[('name', 'Portugal'), ('previousCurrencies', 'Portuguese dinheiro, Portuguese real, escudo')]
[('name', 'Slovakia'), ('previousCurrencies', 'Czechoslovak koruna')]
[('name', 'Slovenia'), ('previousCurrencies', 'Yugoslav dinar')]
[('name', 'Spain'), ('previousCurrencies', 'Haiti

## Task 5
Return all types of dollar in the world (US Dollar, Australian Dollar, ecc...)

In [33]:
# FINAL QUERY
# All the currencies that contain dollar in their name, past and present ones. increased limit 
queryString = """
SELECT DISTINCT ?currency ?currencyName where {

?s wdt:P38 ?currency.
?s sc:name ?name.
?currency sc:name ?currencyName.

FILTER(REGEX(str(?currencyName), ".*(?i)dollar.*")).
}
ORDER BY ?name
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q209792'), ('currencyName', 'Jamaican dollar')]
[('currency', 'http://www.wikidata.org/entity/Q194351'), ('currencyName', 'Barbadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q275112'), ('currencyName', 'Belize dollar')]
[('currency', 'http://www.wikidata.org/entity/Q210478'), ('currencyName', 'Bermudian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q726666'), ('currencyName', 'Mongolian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q942092'), ('currencyName', 'British West Indies dollar')]
[('currency', 'http://www.wikidata.org/entity/Q352796'), ('currencyName', 'trade dollar')]
[('currency', 'http://www.wikidata.org/entity/Q206319'), ('currencyName', 'Brunei dollar')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar')]
[('currency', 'http://www.wikidata.org/entity/Q319885'), ('currencyName', 'Cayman Islands dollar')]
[('currency', 'http://www.wikid

>Results in the format "Currency IRI" - "Currency Label"

In [34]:
# what previous dollars are instances of. Tested with limit 1000
# wrong because printing instances of something else ("?s wdt:P31 ?instance." instead of "?currency wdt:P31 ?instance.")
queryString = """
SELECT DISTINCT ?instance ?instanceName where {

?s wdt:P38 ?currency.
?s wdt:P31 ?instance.
?instance sc:name ?instanceName. 
?s sc:name ?name.
?currency sc:name ?currencyName.

FILTER(REGEX(str(?currencyName), ".*(?i)dollar.*")).
}
ORDER BY ?name
LIMIT 1
#LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q3957'), ('instanceName', 'town')]
1


In [35]:
# what previous dollars are instances of. Tested with limit 1000
queryString = """
SELECT DISTINCT ?instance ?instanceName where {

?s wdt:P38 ?currency.
?currency wdt:P31 ?instance.
?instance sc:name ?instanceName. 
?s sc:name ?name.
?currency sc:name ?currencyName.

FILTER(REGEX(str(?currencyName), ".*(?i)dollar.*")).
}
ORDER BY ?name

LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q610038'), ('instanceName', 'silver coin')]
[('instance', 'http://www.wikidata.org/entity/Q476883'), ('instanceName', 'trade coin')]
[('instance', 'http://www.wikidata.org/entity/Q1643989'), ('instanceName', 'legal tender')]
[('instance', 'http://www.wikidata.org/entity/Q28795800'), ('instanceName', 'decimal currency')]
[('instance', 'http://www.wikidata.org/entity/Q3020761'), ('instanceName', 'Japanese invasion money')]
[('instance', 'http://www.wikidata.org/entity/Q14083'), ('instanceName', 'dollar')]
[('instance', 'http://www.wikidata.org/entity/Q8142'), ('instanceName', 'currency')]
[('instance', 'http://www.wikidata.org/entity/Q756202'), ('instanceName', 'reserve currency')]
[('instance', 'http://www.wikidata.org/entity/Q25530138'), ('instanceName', 'yuan')]
[('instance', 'http://www.wikidata.org/entity/Q28783456'), ('instanceName', 'obsolete currency')]
[('instance', 'http://www.wikidata.org/entity/Q4112083'), ('instanceName',

In [36]:
# All instances of dollar
# I like more the result from the first query for this task because I got more results 
# and all have "dollar" in their name. 
queryString = """
SELECT DISTINCT ?currency ?currencyName where {

?currency wdt:P31 wd:Q14083.
?currency sc:name ?currencyName.

}

LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q209792'), ('currencyName', 'Jamaican dollar')]
[('currency', 'http://www.wikidata.org/entity/Q392858'), ('currencyName', 'Rhodesian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q1104069'), ('currencyName', 'Canadian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q259502'), ('currencyName', 'Australian dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4597'), ('currencyName', 'Solomon Islands dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4917'), ('currencyName', 'United States dollar')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar')]
[('currency', 'http://www.wikidata.org/entity/Q4817670'), ('currencyName', 'Coins and postage stamps of Sealand')]
[('currency', 'http://www.wikidata.org/entity/Q2411666'), ('currencyName', 'Newfoundland dollar')]
[('currency', 'http://www.wikidata.org/entity/Q242988'), ('currencyName', 'Liberian dollar')]
[('cur

## Task 6
Return the currencies used in at least 2 different countries (return also the number of countries using each currency).

In [37]:
# considering all entities that have a currency, but getting also old and not used currencies.. 
queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?s) AS ?numberOfRegions) (GROUP_CONCAT(DISTINCT ?name ; separator =" ,") as ?listOfRegions) where {
?s wdt:P38 ?currency.

?s sc:name ?name.
?currency sc:name ?currencyName.
}
GROUP BY ?currency ?currencyName
HAVING (COUNT(DISTINCT ?s) > 1)
#LIMIT 2
#LIMIT 1000


"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q208008'), ('currencyName', 'cocoa bean'), ('numberOfRegions', '2'), ('listOfRegions', 'Aztec ,Tabasco')]
[('currency', 'http://www.wikidata.org/entity/Q81893'), ('currencyName', 'hryvnia'), ('numberOfRegions', '26'), ('listOfRegions', "Balažer ,Birkivka ,Borok (Koriukivs'kii raion) ,Gurinivka ,Kalinivka ,Klitenka ,Krasnopilka ,Moisiyivka ,Novi Krivotuli ,Novorossiya ,Piatnitske ,Plai ,Republic of Crimea ,Rivne ,Sambir ,Skypche ,Sosnivka ,Starostav ,Sviatohorivka ,Svitlohirske ,Ukraine ,Velikii Khodak ,Veselii Khutir ,Zarichani ,Zoriane")]
[('currency', 'http://www.wikidata.org/entity/Q2361916'), ('currencyName', 'som'), ('numberOfRegions', '2'), ('listOfRegions', 'Second East Turkestan Republic ,Volga Bulgaria')]
[('currency', 'http://www.wikidata.org/entity/Q200192'), ('currencyName', 'Moroccan dirham'), ('numberOfRegions', '9'), ('listOfRegions', 'Arab League ,Bank Al-Maghrib ,Driouch Province ,Ijermaouas ,Morocco ,Oulad Boubker 

In [38]:
# To limit the number of countries since many of the previous ones don't exist anymore, look for another property to use.
# Check if there is any edge from "euro" to countries.
# The intresting predicate is (wdt:P17), with label "country".
# Same one I was using before..
queryString = """
SELECT DISTINCT ?p ?propertyName ?obj ?objName  where {
wd:Q4916 ?p ?obj. 
?p sc:name ?propertyName.
?obj sc:name ?objName.

FILTER(!isLiteral(?obj))
}

LIMIT 100


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('propertyName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q16068'), ('objName', 'Deutsche Mark')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('propertyName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q788472'), ('objName', 'Dutch guilder')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('propertyName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q203354'), ('objName', 'Finnish markka')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('propertyName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q270642'), ('objName', 'Irish pound')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('propertyName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28795800'), ('objName', 'decimal currency')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('propertyName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q7778928'), ('objName', 'Category

In [39]:
# FINAL QUERY
# consider only countries predicate (wdt:P17)
# return the currency and number of countries.
queryString = """
SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?country) as ?countries) where {
?s wdt:P38 ?currency.
?currency wdt:P17 ?country.

?s sc:name ?name.
?currency sc:name ?currencyName.
?country sc:name ?countryName.
#FILTER(COUNT(?currencyName)>1)
}
GROUP BY ?currency ?currencyName
HAVING (COUNT(DISTINCT ?country) > 1)
LIMIT 100
#LIMIT 1000


"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q2616582'), ('currencyName', 'Rhodesian pound'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1246307'), ('currencyName', 'Tornesel'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q618653'), ('currencyName', 'North Yemeni rial'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1852872'), ('currencyName', 'New Hebrides franc'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q2273865'), ('currencyName', 'Italian Somaliland somalo'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q1072248'), ('currencyName', 'Rwanda and Burundi franc'), ('countries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q31015'), ('currencyName', 'Hong Kong dollar'), ('countries', '2')]
[('currency', 'http://www.wikidata.org/entity/Q615640'), ('currencyName', 'Soviet ruble'), ('countries', '3')]
[('currency', 'http://www.wikidata.org/entity/Q10366257'), ('

>Results are in the format "Currency IRI" - "Currency Label" - "Number of using countries"

## Task 7
Consider the currencies obtained in task 6, for each currency return the number of people using that currency (hint: consider the maximum popolation available for each country).

In [40]:
# FINAL QUERY
# starting from the previous query find the max population for each country and return the sum over all countries sharing 
# the same currency. #I might be summing populations of countries not existing anymore..
queryString = """
SELECT DISTINCT ?currency ?currencyName (SUM(?maxPopulationPerCountry) AS ?population) where{
    ?currency sc:name ?currencyName.
    
    {
        SELECT DISTINCT ?currency (MAX(?o) as ?maxPopulationPerCountry) ?country ?countryName where{
                        ?currency wdt:P17 ?country.
                        ?country ?p ?o.

                        ?p sc:name ?pName.
                        ?country sc:name ?countryName.

                        FILTER (REGEX(?pName,".*(?i)population.*")) #it includes wdt:P1082

            {
            SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?countries) where {
                            ?s wdt:P38 ?currency.
                            ?currency wdt:P17 ?country.

                            ?s sc:name ?name.
                            ?currency sc:name ?currencyName.
                            ?country sc:name ?countryName.

                }
            GROUP BY ?currency ?currencyName
            HAVING (COUNT(DISTINCT ?country) > 1)
            }

        }
GROUP BY ?currency ?pName ?country ?countryName
    }
}


"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc'), ('population', '67180549')]
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('population', '8503939')]
[('currency', 'http://www.wikidata.org/entity/Q2273865'), ('currencyName', 'Italian Somaliland somalo'), ('population', '11031386')]
[('currency', 'http://www.wikidata.org/entity/Q3973271'), ('currencyName', 'Palestine pound'), ('population', '10857520')]
[('currency', 'http://www.wikidata.org/entity/Q2616582'), ('currencyName', 'Rhodesian pound'), ('population', '6930000')]
[('currency', 'http://www.wikidata.org/entity/Q11220294'), ('currencyName', 'gold dinar'), ('population', '70000000')]
[('currency', 'http://www.wikidata.org/entity/Q373429'), ('currencyName', 'Yugoslav dinar'), ('population', '34384898')]
[('currency', 'http://www.wikidata.org/entity/Q189097'), ('currencyName', 'peseta'), ('population', '48660878')]
[('currency', 'http://www.

>Results in format "Currency IRI" - "Currency Label" - "Total Aggregated Population"

In [41]:
#Same as previous query, but considering all possible regions that have a currency
queryString = """
SELECT DISTINCT ?currency ?currencyName (SUM(?maxPopulationPerCountry) AS ?population) where{
    ?currency sc:name ?currencyName.
    
    {
        SELECT DISTINCT ?currency (MAX(?o) as ?maxPopulationPerCountry) ?country ?countryName where{
                        ?currency wdt:P17 ?country.
                        ?country ?p ?o.

                        ?p sc:name ?pName.
                        ?country sc:name ?countryName.

                        FILTER (REGEX(?pName,".*(?i)population.*"))

            {
            SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?s) AS ?countries) where {
                            ?s wdt:P38 ?currency.

                            ?currency sc:name ?currencyName.
                            ?s sc:name ?countryName.

                }
            GROUP BY ?currency ?currencyName
            HAVING (COUNT(DISTINCT ?s) > 1)
            }

        }
GROUP BY ?currency ?pName ?country ?countryName
    }
}

LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q184172'), ('currencyName', 'French franc'), ('population', '67180549')]
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('population', '8503939')]
[('currency', 'http://www.wikidata.org/entity/Q2273865'), ('currencyName', 'Italian Somaliland somalo'), ('population', '11031386')]
[('currency', 'http://www.wikidata.org/entity/Q39099'), ('currencyName', 'renminbi'), ('population', '1443497378')]
[('currency', 'http://www.wikidata.org/entity/Q897424'), ('currencyName', 'Hungarian korona'), ('population', '9769526')]
[('currency', 'http://www.wikidata.org/entity/Q203567'), ('currencyName', 'Nigerian naira'), ('population', '190886311')]
[('currency', 'http://www.wikidata.org/entity/Q2616582'), ('currencyName', 'Rhodesian pound'), ('population', '6930000')]
[('currency', 'http://www.wikidata.org/entity/Q123213'), ('currencyName', 'złoty'), ('population', '38382576')]
[('currency', 'http://www.wiki

In [42]:
# In the following I want to check if I can return just currencies currently in use.
# Look if currencies have a start/end date -> at least some have (CTRL+F). 
# I am interested only in the end time (wdt:P582)
queryString = """
SELECT DISTINCT ?p ?name where {
?s wdt:P38 ?currency.
?currency ?p ?obj.

?currency sc:name ?currencyName.
?p sc:name ?name.
FILTER(isLiteral(?obj))

}
GROUP BY ?currency ?currencyName
LIMIT 1000
#LIMIT 1000


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P2968'), ('name', 'QUDT unit ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3365'), ('name', 'Treccani ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7818'), ('name', 'French Vikidia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4212'), ('name', 'PACTOLS thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P498'), ('name', 'ISO 4217 code')]
[('p', 'http://www.wikidata.org/prop/direct/P5061'), ('name', 'unit symbol')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'st

In [43]:
# In the following I want to check if I can return just countries currently existing.
# I am interested only in the end time (wdt:P582)
# Not very efficient, i could have checked just the property wdt:P582 to exist
# Next query is the same as this one, just a little more efficient

queryString = """
SELECT DISTINCT ?p ?name where {
?s wdt:P38 ?currency.
?currency wdt:P17 ?country.
?country ?p ?obj.

?p sc:name ?name.
?currency sc:name ?currencyName.
?country sc:name ?countryName.
FILTER(REGEX(?name,"(?i).*end.*"))
}
LIMIT 1000
#LIMIT 1000


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5698'), ('name', 'Invasive Species Compendium Datasheet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P7295'), ('name', 'Gregorian calendar start date')]
[('p', 'http://www.wikidata.org/prop/direct/P2402'), ('name', 'total expenditure')]
[('p', 'http://www.wikidata.org/prop/direct/P7284'), ('name', 'The Independent topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
6


In [44]:
# Same as the previous query, but considering only properties with literal objects.
# Added FILTER(isLiteral(?obj))

queryString = """
SELECT DISTINCT ?p ?name where {
?s wdt:P38 ?currency.
?currency wdt:P17 ?country.
?country ?p ?obj.

?p sc:name ?name.
?currency sc:name ?currencyName.
?country sc:name ?countryName.
FILTER(isLiteral(?obj))
FILTER(REGEX(?name,"(?i).*end.*"))

}
LIMIT 1000
#LIMIT 1000


"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5698'), ('name', 'Invasive Species Compendium Datasheet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P7295'), ('name', 'Gregorian calendar start date')]
[('p', 'http://www.wikidata.org/prop/direct/P2402'), ('name', 'total expenditure')]
[('p', 'http://www.wikidata.org/prop/direct/P7284'), ('name', 'The Independent topic ID')]
5


In [45]:
# This is the same query as the first query for Task 7, but considering only currently existing countries and in use currencies, using predicate "end time"(wdt:P582)
# If no end time, I suppose the currency/country to exist.
# Still some currencies from the result set are not used anymore. 
queryString = """
SELECT DISTINCT ?currency ?currencyName (SUM(?maxPopulationPerCountry) AS ?population) where{
    ?currency sc:name ?currencyName.
    FILTER NOT EXISTS {?currency wdt:P582 ?endTime.} #no currency end time -> currency still in use
    FILTER NOT EXISTS {?country wdt:P582 ?endTime1} #not existing country -> consider only existing countries
    
    {
        SELECT DISTINCT ?currency (MAX(?o) as ?maxPopulationPerCountry) ?country ?countryName where{
                        ?currency wdt:P17 ?country.
                        ?country ?p ?o.

                        ?p sc:name ?pName.
                        ?country sc:name ?countryName.

                        FILTER (REGEX(?pName,".*(?i)population.*"))

            {
            SELECT DISTINCT ?currency ?currencyName (COUNT(DISTINCT ?country) AS ?countries) where {
                            ?s wdt:P38 ?currency.
                            ?currency wdt:P17 ?country.

                            ?s sc:name ?name.
                            ?currency sc:name ?currencyName.
                            ?country sc:name ?countryName.

                }
            GROUP BY ?currency ?currencyName
            HAVING (COUNT(DISTINCT ?country) > 1)
            }

        }
GROUP BY ?currency ?pName ?country ?countryName
    }
}
LIMIT 300

"""

print("Results")
x=run_query(queryString)

Results
[('currency', 'http://www.wikidata.org/entity/Q25344'), ('currencyName', 'Swiss franc'), ('population', '8503939')]
[('currency', 'http://www.wikidata.org/entity/Q2616582'), ('currencyName', 'Rhodesian pound'), ('population', '6930000')]
[('currency', 'http://www.wikidata.org/entity/Q11220294'), ('currencyName', 'gold dinar'), ('population', '70000000')]
[('currency', 'http://www.wikidata.org/entity/Q26365'), ('currencyName', 'Eastern Caribbean dollar'), ('population', '647235')]
[('currency', 'http://www.wikidata.org/entity/Q522701'), ('currencyName', 'Netherlands Antillean guilder'), ('population', '197469')]
[('currency', 'http://www.wikidata.org/entity/Q34174'), ('currencyName', 'Tarifit'), ('population', '124080318')]
[('currency', 'http://www.wikidata.org/entity/Q554157'), ('currencyName', 'Reichsthaler'), ('population', '18981215')]
[('currency', 'http://www.wikidata.org/entity/Q306306'), ('currencyName', 'Austro-Hungarian gulden'), ('population', '74000000')]
[('currenc